In [1]:
import os
import argparse
import torch
import numpy as np
from tqdm import tqdm
from scipy.io import wavfile

from torch.multiprocessing import Pool, set_start_method


from ttsxai.utils.utils import read_ljs_metadata
from ttsxai.interfaces.tts_interface import TTSInterface, get_text2mel, get_mel2wave



In [2]:
from ttsxai.utils import jupyter_utils
jupyter_utils.register_always_reload(packages=[
    # 'ttsxai.interfaces',
    # 'ttsxai.models.text2mel',
    # 'ttsxai.models.mel2wave',
    # 'ttsxai.utils',
])
jupyter_utils.register_always_reload(modules=[
    # 'ttsxai.text',
    # 'ttsxai.models.text2mel.tacotron2',
    # 'ttsxai.models.text2mel.fastspeech2',
    'ttsxai.utils.utils',
    # 'ttsxai.pitch_calculator',
    # 'ttsxai.energy_calculator'
])

In [3]:
PACKAGE_DIR

'/nas/users/dahye/kw/tts/ttsxai'

In [ ]:
text_dict = read_ljs_metafile(os.path.join(args.data_dir, 'metadata.csv'))

In [4]:
def read_ljs_metafile(path):
    text_dict = {}
    with open(path, encoding='utf-8') as f:
        for line in f:
            split = line.split('|')
            text_id, text = split[0], split[-1]
            text_dict[text_id] = text.strip()
    return text_dict

In [14]:
read_ljs_metafile(os.path.join(PACKAGE_DIR, 'src/ttsxai/filelists/ljs_audio_text_train_filelist.txt'))

{'LJ001-0001': 'Printing, in the only sense with which we are at present concerned, differs from most if not from all the arts and crafts represented in the Exhibition',
 'LJ001-0002': 'in being comparatively modern.',
 'LJ001-0003': 'For although the Chinese took impressions from wood blocks engraved in relief for centuries before the woodcutters of the Netherlands, by a similar process',
 'LJ001-0004': 'produced the block books, which were the immediate predecessors of the true printed book,',
 'LJ001-0005': 'the invention of movable metal letters in the middle of the fifteenth century may justly be considered as the invention of the art of printing.',
 'LJ001-0006': 'And it is worth mention in passing that, as an example of fine typography,',
 'LJ001-0007': 'the earliest book printed with movable types, the Gutenberg, or "forty-two line Bible" of about fourteen fifty-five,',
 'LJ001-0008': 'has never been surpassed.',
 'LJ001-0009': 'Printing, then, for our purpose, may be considere

In [16]:
data_dir = "/nas/users/dahye/kw/tts/ttsxai/data/LJSpeech"
read_ljs_metadata(mode='val')

{'LJ001-0110': 'Even the Caslon type when enlarged shows great shortcomings in this respect:',
 'LJ002-0018': 'The inadequacy of the jail was noticed and reported upon again and again by the grand juries of the city of London,',
 'LJ002-0043': 'long narrow rooms -- one thirty-six feet, six twenty-three feet, and the eighth eighteen,',
 'LJ003-0111': 'He was in consequence put out of the protection of their internal law, end quote. Their code was a subject of some curiosity.',
 'LJ003-0345': 'All the committee could do in this respect was to throw the responsibility on others.',
 'LJ004-0045': 'Mr. Sturges Bourne, Sir James Mackintosh, Sir James Scarlett, and William Wilberforce.',
 'LJ004-0096': 'the fatal consequences whereof might be prevented if the justices of the peace were duly authorized',
 'LJ004-0152': "although at Mr. Buxton's visit a new jail was in process of erection, the first step towards reform since Howard's visitation in seventeen seventy-four.",
 'LJ005-0014': 'Speak

In [15]:
import glob
data_activation_dir = "/nas/users/dahye/kw/tts/ttsxai/data_activation/LJSpeech/tacotron2_waveglow"
glob.glob(os.path.join(data_activation_dir, '*.npz'))

['/nas/users/dahye/kw/tts/ttsxai/data_activation/LJSpeech/tacotron2_waveglow/LJ037-0213.npz',
 '/nas/users/dahye/kw/tts/ttsxai/data_activation/LJSpeech/tacotron2_waveglow/LJ023-0097.npz',
 '/nas/users/dahye/kw/tts/ttsxai/data_activation/LJSpeech/tacotron2_waveglow/LJ012-0075.npz',
 '/nas/users/dahye/kw/tts/ttsxai/data_activation/LJSpeech/tacotron2_waveglow/LJ023-0098.npz',
 '/nas/users/dahye/kw/tts/ttsxai/data_activation/LJSpeech/tacotron2_waveglow/LJ001-0001.npz',
 '/nas/users/dahye/kw/tts/ttsxai/data_activation/LJSpeech/tacotron2_waveglow/LJ037-0214.npz',
 '/nas/users/dahye/kw/tts/ttsxai/data_activation/LJSpeech/tacotron2_waveglow/LJ001-0002.npz',
 '/nas/users/dahye/kw/tts/ttsxai/data_activation/LJSpeech/tacotron2_waveglow/LJ012-0076.npz',
 '/nas/users/dahye/kw/tts/ttsxai/data_activation/LJSpeech/tacotron2_waveglow/LJ023-0099.npz',
 '/nas/users/dahye/kw/tts/ttsxai/data_activation/LJSpeech/tacotron2_waveglow/LJ037-0215.npz',
 '/nas/users/dahye/kw/tts/ttsxai/data_activation/LJSpeech/ta